In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
#train_dataset = datasets.MNIST(root='/root/data/', train=True, transform=transforms.ToTensor(), download=True)
train_dataset = datasets.MNIST(root='/root/data/', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = datasets.MNIST(root='/root/data/', train=False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [18]:
class ResidualBlock(nn.Module):

    def __init__(self, dim):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(dim, dim, 3, 1, 1)
        self.conv2 = nn.Conv2d(dim, dim, 1)

    def forward(self, x):
        tmp = self.relu(x)
        tmp = self.conv1(tmp)
        tmp = self.relu(tmp)
        tmp = self.conv2(tmp)
        return x + tmp


class VQVAE(nn.Module):

    def __init__(self, input_dim, dim, n_embedding):
        super().__init__()
        self.encoder = nn.Sequential(nn.Conv2d(input_dim, dim[0], 4, 2, 1),
                                     nn.ReLU(), nn.Conv2d(dim[0], dim[1], 4, 2, 1),
                                     nn.ReLU(), nn.Conv2d(dim[1], dim[2], 3, 1, 1),
                                     ResidualBlock(dim[2]), ResidualBlock(dim[2]))
        self.vq_embedding = nn.Embedding(n_embedding, dim[2])
        self.vq_embedding.weight.data.uniform_(-1.0 / n_embedding,
                                               1.0 / n_embedding)
        self.decoder = nn.Sequential(
            nn.Conv2d(dim[2], dim[1], 3, 1, 1),
            ResidualBlock(dim[1]), ResidualBlock(dim[1]),
            nn.ConvTranspose2d(dim[1], dim[0], 4, 2, 1), nn.ReLU(),
            nn.ConvTranspose2d(dim[0], input_dim, 4, 2, 1))
        self.n_downsample = 2

    def forward(self, x):
        # encode
        ze = self.encoder(x)

        # ze: [N, C, H, W]
        # embedding [K, C]
        embedding = self.vq_embedding.weight.data
        N, C, H, W = ze.shape
        K, _ = embedding.shape
        embedding_broadcast = embedding.reshape(1, K, C, 1, 1)
        ze_broadcast = ze.reshape(N, 1, C, H, W)
        distance = torch.sum((embedding_broadcast - ze_broadcast)**2, 2)
        nearest_neighbor = torch.argmin(distance, 1)
        # make C to the second dim
        zq = self.vq_embedding(nearest_neighbor).permute(0, 3, 1, 2)
        # stop gradient
        decoder_input = ze + (zq - ze).detach()

        # decode
        x_hat = self.decoder(decoder_input)
        return x_hat, ze, zq

    @torch.no_grad()
    def encode(self, x):
        ze = self.encoder(x)
        embedding = self.vq_embedding.weight.data

        # ze: [N, C, H, W]
        # embedding [K, C]
        N, C, H, W = ze.shape
        K, _ = embedding.shape
        embedding_broadcast = embedding.reshape(1, K, C, 1, 1)
        ze_broadcast = ze.reshape(N, 1, C, H, W)
        distance = torch.sum((embedding_broadcast - ze_broadcast)**2, 2)
        nearest_neighbor = torch.argmin(distance, 1)
        return nearest_neighbor

    @torch.no_grad()
    def decode(self, discrete_latent):
        zq = self.vq_embedding(discrete_latent).permute(0, 3, 1, 2)
        x_hat = self.decoder(zq)
        return x_hat

    # Shape: [C, H, W]
    def get_latent_HW(self, input_shape):
        C, H, W = input_shape
        return (H // 2**self.n_downsample, W // 2**self.n_downsample)

In [19]:
from time import time

n_epochs = 20
learning_rate = 1e-2
alpha = 1
beta = 0.25

pi = torch.tensor(torch.pi)

def loss_fn(x, x_hat, ze, zq):
    l_reconstruct = F.mse_loss(x, x_hat)
    l_1 = F.mse_loss(ze.detach(), zq)
    l_2 = F.mse_loss(ze, zq.detach())
    loss = l_reconstruct + alpha * l_1 + beta * l_2
    return loss/len(x)

def loss_reconstruction(x, x_hat):
    return F.mse_loss(x, x_hat)/len(x)

def loss_emb(ze, zq):
    l_1 = F.mse_loss(ze.detach(), zq)
    l_2 = F.mse_loss(ze, zq.detach())
    return (alpha * l_1 + beta * l_2)/len(ze)

def train(device, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    begin_time = time()
    # train
    with open('/root/new/VQVAE/pix/loss.txt', 'w') as file:
        for i in range(n_epochs):
            for batch_idx, (x, _) in enumerate(train_loader):
                x = x.to(device)
                x_hat, ze, zq = model(x)
                loss = loss_fn(x, x_hat, ze, zq)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            # estimate loss
            model.eval()
            with torch.no_grad():
                each_epoch = 10
                n_samples = 100
                #train
                indices = torch.randperm(len(train_dataset))[:n_samples]
                x = torch.stack([train_dataset[i][0] for i in indices]).to(device)
                x_hat, ze, zq = model(x)
                loss = loss_fn(x, x_hat, ze, zq)
                loss_recons = loss_reconstruction(x, x_hat)
                loss_embed = loss_emb(ze, zq)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.7f}'.format(i, loss))
                    print('====> Epoch: {} reconst loss: {:.7f}'.format(i, loss_recons))
                    print('====> Epoch: {} embedding loss: {:.7f}'.format(i, loss_embed))
                file.write(str(loss.item()) + ' ' + str(loss_recons.item()) + ' ' + str(loss_embed.item()) + ' ')
                #test
                indices = torch.randperm(len(test_dataset))[:n_samples]
                x = torch.stack([test_dataset[i][0] for i in indices]).to(device)
                x_hat, ze, zq = model(x)
                loss = loss_fn(x, x_hat, ze, zq)
                loss_recons = loss_reconstruction(x, x_hat)
                loss_embed = loss_emb(ze, zq)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.7f}'.format(i, loss))
                    print('====> Epoch: {} reconst loss: {:.7f}'.format(i, loss_recons))
                    print('====> Epoch: {} embedding loss: {:.7f}'.format(i, loss_embed))
                file.write(str(loss.item()) + ' ' + str(loss_recons.item()) + ' ' + str(loss_embed.item()) + '\n')
            
            #time
            if(i % each_epoch == 0):
                training_time = time() - begin_time
                minute = int(training_time // 60)
                second = int(training_time % 60)
                print(f'time loss {minute}:{second}')
        
        torch.save(model, '/root/new/VQVAE/pix/model.pth')
        
    tot_training_time = time() - begin_time
    minute = int(tot_training_time // 60)
    second = int(tot_training_time % 60)
    print(f'total time loss {minute}:{second}')

def initialize_parameters(model):
    for param in model.parameters():
        param.data.normal_(mean=0, std=0.01)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
        
import numpy as np
import pandas as pd

In [ ]:
def main():
    device = 'cuda:0'

    model = VQVAE(input_dim=1, dim=[32, 128, 2], n_embedding=10).to(device)

    # Load the model
    #model.load_state_dict(torch.load('/root/new/VQVAE/model.pth', 'cuda:0'))

    #Choose which to play
    train(device, model)
    #generate(device, model)
    #reconstruct(device, model)
    #latent_space(device, model)
    #print(sum(p.numel() for p in model.parameters() if p.requires_grad))

if __name__ == '__main__':
    main()

In [ ]:
device = 'cpu'
model = torch.load('/root/new/VQVAE/pix/model.pth', map_location=device)
# Reconstruction
with torch.no_grad(): 
    
    for batch_idx, (batch_x, _) in enumerate(test_loader):
        true_imgs = batch_x.view(-1, 1, 28, 28)
        save_image(true_imgs, '/root/new/VQVAE/pix/pictures/oring.png', nrow=10)
        break
    
    x = true_imgs.to(device)
    
    reconst_x = model(x)[0]
    reconst_imgs = reconst_x.view(-1, 1, 28, 28)
    print(torch.max(reconst_imgs))
    print(torch.min(reconst_imgs))
    reconst_imgs = torch.clamp(reconst_imgs, max=1)
    print(F.mse_loss(true_imgs, reconst_imgs).item()/len(true_imgs))
    #print(torch.nn.functional.binary_cross_entropy(reconst_imgs, true_imgs, reduction='sum').item()/len(true_imgs))
    save_image(reconst_imgs, '/root/new/VQVAE/pix/pictures/recons.png', nrow=10)